In [1]:
%load_ext autoreload

In [2]:
import sys
sys.path.insert(0, '/home/divanik/CourseWork/Nystrom_TT_rounding_and_convolution') # Path to repository root

In [3]:
import numpy as np
import tt
from algorithms_package.src import primitives
from algorithms_package.src.hadamard_product import generalizedApproximateHadamardProduct
from algorithms_package.src.convolution import (
    approximateConvolution,
    preciseConvolution,
    countFourier,
    countInverseFourier,
    approximateCycleConvolution,
    approximateTwoSidedCycleConvolution
)
import time


from newton_experiment_primitives import create_exponential_grid, create_newtonial_potential_grid

from algorithms_package.src.convolution import preciseCycleConvolution
from newton_experiment_primitives import interpolateTTTensor, compressTTTensor
import pandas as pd
import numpy as np

In [4]:
D = 40

In [5]:
# for d in range(4, 7):
#     for grid_side in np.logspace(6, 9, 7, base=2, dtype=int):
#         grid_side = 8 * (grid_side // 8)
#         print(grid_side)
#         h = (2 * D) / grid_side
#         tensor1 = create_exponential_grid(d, grid_side + 2, D + h / 2, eps=1e-12)
#         tensor2 = create_newtonial_potential_grid(d, 2 * grid_side + 2, h * grid_side + h / 2, eps=1e-12)
#         tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(d)])
#         print(tt.vector.from_list(tensor1))
#         print(tt.vector.from_list(tensor2))
#         precise = preciseCycleConvolution(tensor1, tensor2)
#         precise = primitives.twoSidedCuttingTTTensor(precise, [(0, grid_side + 1) for _ in range(d)])
#         rounded = tt.vector.from_list(precise).round(0, 180)
#         print(tt.vector.norm(tt.vector.from_list(precise) - rounded))
#         norm = tt.vector.norm(rounded)
#         del(precise)
#         for rank in range(10, 31, 4):
#             convolution = approximateTwoSidedCycleConvolution(tensor1, tensor2, 5 * [rank], 5 * [rank], 10, True)
#             convolution = primitives.twoSidedCuttingTTTensor(convolution, [(0, grid_side + 1) for _ in range(d)])
#             print(tt.vector.norm(tt.vector.from_list(convolution) - rounded) / norm)

In [6]:
current_ranks = {
    3 : range(10, 31, 4),
    4 : range(10, 41, 4)
}

In [7]:
answer = []

for d in range(3, 5):
    grid_side = 300
    h = (2 * D) / grid_side
    print("kek")
    tensor1 = create_exponential_grid(d, grid_side + 2, D + h / 2, eps=1e-12)
    tensor2 = create_newtonial_potential_grid(d, 2 * grid_side + 2, h * grid_side + h / 2, eps=1e-12)
    tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(d)])
    precise = preciseCycleConvolution(tensor1, tensor2)
    precise = primitives.twoSidedCuttingTTTensor(precise, [(0, grid_side + 1) for _ in range(d)])
    rounded = tt.vector.from_list(precise).round(0, 100)
    norm = tt.vector.norm(rounded)
    del(precise)
    print("Skip: start algos:")
    for oversampling in [0, 1, 3, 5, 7]:
        for rank in current_ranks[d]:
            rounding_rank = int(rank * 1.5)
            convolution = approximateTwoSidedCycleConvolution(tensor1, tensor2, (d - 1) * [rank + oversampling], (d - 1) * [rounding_rank + oversampling], 1000000 * d + 1000 * oversampling + rank, True)
            convolution = tt.vector.to_list(tt.vector.from_list(convolution).round(0, rank))
            convolution = primitives.twoSidedCuttingTTTensor(convolution, [(0, grid_side + 1) for _ in range(d)])
            precision = tt.vector.norm(tt.vector.from_list(convolution) - rounded) / norm
            answer.append({'d' : d,'oversampling' : oversampling, 'rank' : rank, 'precision': precision})
            print(d, oversampling, rank, precision)

kek
Skip: start algos:
3 0 10 9.486285865183885e-05
3 0 14 9.375340572724774e-06
3 0 18 2.315115929754938e-07
3 0 22 1.6443935890699065e-08
3 0 26 1.0122824087492077e-08
3 0 30 1.008803148369878e-08
3 1 10 5.928674588010912e-05
3 1 14 3.231908837808351e-06
3 1 18 1.3771863784758037e-07
3 1 22 1.2215625772787854e-08
3 1 26 1.0140837653580532e-08
3 1 30 1.0092635277688936e-08
3 3 10 2.0387448427030395e-05
3 3 14 9.073405611194495e-07
3 3 18 7.251384364373282e-08
3 3 22 1.0427235382882483e-08
3 3 26 1.0090697409885219e-08
3 3 30 1.008863041037763e-08
3 5 10 1.9774783355523854e-05
3 5 14 9.106871234549841e-07
3 5 18 3.605664716638592e-08
3 5 22 1.0536317563463993e-08
3 5 26 1.0101962547800972e-08
3 5 30 1.0089298278896466e-08
3 7 10 1.975202384404048e-05
3 7 14 8.771381196582047e-07
3 7 18 3.660462855312191e-08
3 7 22 1.0286982464992743e-08
3 7 26 1.0089877007143153e-08
3 7 30 1.0089367118993441e-08
kek
Skip: start algos:
4 0 10 0.0007112113520795678
4 0 14 4.964263137028495e-05
4 0 18 3.7